In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import random
import math

game_over = False
game_score = 0
life = 5
bullets_missed = 0

gun_pos = [0, 0, 0]
gun_angle = 0
gun_speed = 20  
rotation_speed = 15 


camera_angle = 0
camera_height = 500
camera_distance = 500
first_person_mode = False


cheat_mode = False
cheat_vision = False
auto_rotate_angle = 0

bullets = []
bullet_speed = 20  


enemies = []
enemy_speed = 1
enemy_size_scale = 1.0
enemy_size_growing = True

# Game constants
GRID_LENGTH = 600
BOUNDARY_HEIGHT = 100
MAX_BULLETS_MISSED = 10
fovY = 60


class Bullet:
    def __init__(self, x, y, angle):
        self.x = x
        self.y = y
        self.z = 30
        self.angle = angle
        self.active = True
        self.size = 8
        
    def update(self):
        self.x += bullet_speed * math.sin(math.radians(self.angle))
        self.y += bullet_speed * math.cos(math.radians(self.angle))
        
        if abs(self.x) > GRID_LENGTH or abs(self.y) > GRID_LENGTH:
            self.active = False
            
    def draw(self):
        if self.active:
            glPushMatrix()
            glTranslatef(self.x, self.y, self.z)
            glColor3f(1, 1, 0)
            glutSolidCube(self.size)
            glPopMatrix()


class Enemy:
    def __init__(self):
        self.respawn()
        
    def respawn(self):
        side = random.randint(0, 3)
        if side == 0:
            self.x = random.uniform(-GRID_LENGTH + 50, GRID_LENGTH - 50)
            self.y = GRID_LENGTH - 50
        elif side == 1:
            self.x = random.uniform(-GRID_LENGTH + 50, GRID_LENGTH - 50)
            self.y = -GRID_LENGTH + 50
        elif side == 2:
            self.x = -GRID_LENGTH + 50
            self.y = random.uniform(-GRID_LENGTH + 50, GRID_LENGTH - 50)
        else:
            self.x = GRID_LENGTH - 50
            self.y = random.uniform(-GRID_LENGTH + 50, GRID_LENGTH - 50)
        
        self.z = 20
        self.radius = 25
        
    def update(self, target_x, target_y):
        dx = target_x - self.x
        dy = target_y - self.y
        distance = math.sqrt(dx*dx + dy*dy)
        
        if distance > 0:
            self.x += (dx / distance) * enemy_speed
            self.y += (dy / distance) * enemy_speed
            
    def draw(self, size_scale):
        glPushMatrix()
        glTranslatef(self.x, self.y, self.z)
        
        glColor3f(0.8, 0.2, 0.2)
        gluSphere(gluNewQuadric(), self.radius * size_scale, 20, 20)
        
        glTranslatef(0, 0, self.radius * size_scale * 1.5)
        glColor3f(1, 0.3, 0.3)
        gluSphere(gluNewQuadric(), self.radius * size_scale * 0.8, 20, 20)
        
        glPopMatrix()
        
    def check_collision_with_bullet(self, bullet):
        dx = self.x - bullet.x
        dy = self.y - bullet.y
        dz = self.z - bullet.z
        distance = math.sqrt(dx*dx + dy*dy + dz*dz)
        return distance < (self.radius + bullet.size)
        
    def check_collision_with_player(self, px, py):
        dx = self.x - px
        dy = self.y - py
        distance = math.sqrt(dx*dx + dy*dy)
        return distance < (self.radius + 30)


def initialize_game():
    global enemies
    enemies = [Enemy() for _ in range(5)]


def draw_text(x, y, text, font=GLUT_BITMAP_HELVETICA_18):
    glColor3f(1, 1, 1)
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, 1000, 0, 800)
    
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadIdentity()
    
    glRasterPos2f(x, y)
    for ch in text:
        glutBitmapCharacter(font, ord(ch))
    
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)


def draw_player(lying_down=False):
    """
    Draw player EXACTLY matching the reference images:
    - Round head sphere at top
    - Thin gun barrel sticking forward
    - Rectangular gray/olive body 
    - Two bright blue cylindrical legs
    """
    glPushMatrix()
    glTranslatef(gun_pos[0], gun_pos[1], gun_pos[2])
    
    if lying_down:
        glRotatef(90, 1, 0, 0)
    
    #Rotate_player around Z-axis
    glRotatef(gun_angle, 0, 0, 1)
    
    #HEAD_Sphere at top
    glPushMatrix()
    glTranslatef(0, 0, 48)
    glColor3f(0.0, 0.0, 0.0)  #black head
    gluSphere(gluNewQuadric(), 12, 20, 20)
    glPopMatrix()
    
    #GUNBARREL_Long thin cylinder
    glPushMatrix()
    glTranslatef(0, 12, 48)  # Start from front of head
    glRotatef(90, 1, 0, 0)  # along +Y axis
    glColor3f(1.0, 1.0, 1.0)  # Almost black
    gluCylinder(gluNewQuadric(), 2.5, 2.5, 45, 12, 12)  # Long thin barrel
    glPopMatrix()
    
    # ========== TORSO/BODY (Cuboid - rectangular box) ==========
    glPushMatrix()
    glTranslatef(0, 0, 28)  # Below head
    glColor3f(0.55, 0.6, 0.5)  # Olive/gray-green body
    glScalef(1.4, 1.0, 2.2)  # Make it rectangular (tall)
    glutSolidCube(16)
    glPopMatrix()
    
    # ========== GUN GRIP/HANDLE (Small cuboid behind gun) ==========
    glPushMatrix()
    glTranslatef(0, -6, 45)
    glColor3f(0.3, 0.3, 0.3)  # Dark gray grip
    glScalef(0.7, 0.9, 0.5)
    glutSolidCube(10)
    glPopMatrix()
    
    # ========== LEFT LEG (Blue cylinder) ==========
    glPushMatrix()
    glTranslatef(-7, 0, 16)  # Left position
    glRotatef(90, 1, 0, 0)  # Point downward
    glColor3f(0.0, 0.0, 0.95)  # Bright blue legs
    gluCylinder(gluNewQuadric(), 5.5, 4.5, 16, 15, 15)  # Slightly tapered
    glPopMatrix()
    
    # ========== RIGHT LEG (Blue cylinder) ==========
    glPushMatrix()
    glTranslatef(7, 0, 16)  # Right position
    glRotatef(90, 1, 0, 0)  # Point downward
    glColor3f(0.0, 0.0, 0.95)  # Bright blue legs
    gluCylinder(gluNewQuadric(), 5.5, 4.5, 16, 15, 15)  # Slightly tapered
    glPopMatrix()
    
    glPopMatrix()


def draw_grid():
    """Draw checkerboard floor dynamically"""
    glBegin(GL_QUADS)
    
    grid_size = 60
    for i in range(-10, 10):
        for j in range(-10, 10):
            if (i + j) % 2 == 0:
                glColor3f(0.9, 0.9, 0.9)
            else:
                glColor3f(0.7, 0.7, 0.7)
            
            x1 = i * grid_size
            y1 = j * grid_size
            x2 = (i + 1) * grid_size
            y2 = (j + 1) * grid_size
            
            glVertex3f(x1, y1, 0)
            glVertex3f(x2, y1, 0)
            glVertex3f(x2, y2, 0)
            glVertex3f(x1, y2, 0)
    
    glEnd()
    
    # Draw boundary walls
    glColor3f(0.3, 0.3, 0.3)
    
    glBegin(GL_QUADS)
    glVertex3f(-GRID_LENGTH, GRID_LENGTH, 0)
    glVertex3f(GRID_LENGTH, GRID_LENGTH, 0)
    glVertex3f(GRID_LENGTH, GRID_LENGTH, BOUNDARY_HEIGHT)
    glVertex3f(-GRID_LENGTH, GRID_LENGTH, BOUNDARY_HEIGHT)
    glEnd()
    
    glBegin(GL_QUADS)
    glVertex3f(-GRID_LENGTH, -GRID_LENGTH, 0)
    glVertex3f(GRID_LENGTH, -GRID_LENGTH, 0)
    glVertex3f(GRID_LENGTH, -GRID_LENGTH, BOUNDARY_HEIGHT)
    glVertex3f(-GRID_LENGTH, -GRID_LENGTH, BOUNDARY_HEIGHT)
    glEnd()
    
    glBegin(GL_QUADS)
    glVertex3f(-GRID_LENGTH, -GRID_LENGTH, 0)
    glVertex3f(-GRID_LENGTH, GRID_LENGTH, 0)
    glVertex3f(-GRID_LENGTH, GRID_LENGTH, BOUNDARY_HEIGHT)
    glVertex3f(-GRID_LENGTH, -GRID_LENGTH, BOUNDARY_HEIGHT)
    glEnd()
    
    glBegin(GL_QUADS)
    glVertex3f(GRID_LENGTH, -GRID_LENGTH, 0)
    glVertex3f(GRID_LENGTH, GRID_LENGTH, 0)
    glVertex3f(GRID_LENGTH, GRID_LENGTH, BOUNDARY_HEIGHT)
    glVertex3f(GRID_LENGTH, -GRID_LENGTH, BOUNDARY_HEIGHT)
    glEnd()


def fire_bullet():
    if not game_over:
        bullet = Bullet(gun_pos[0], gun_pos[1], gun_angle)
        bullets.append(bullet)


def update_game():
    global game_over, game_score, life, bullets_missed, enemy_size_scale, enemy_size_growing
    global auto_rotate_angle
    
    if game_over:
        return
    
    # Enemy pulsing effect
    if enemy_size_growing:
        enemy_size_scale += 0.02  # Faster pulsing
        if enemy_size_scale > 1.2:
            enemy_size_growing = False
    else:
        enemy_size_scale -= 0.02  # Faster pulsing
        if enemy_size_scale < 0.8:
            enemy_size_growing = True
    
    # Update bullets
    for bullet in bullets[:]:
        bullet.update()
        
        hit = False
        for enemy in enemies:
            if enemy.check_collision_with_bullet(bullet):
                bullet.active = False
                enemy.respawn()
                game_score += 1
                hit = True
                break
        
        if not bullet.active:
            if not hit:
                bullets_missed += 1
            bullets.remove(bullet)
    
    # Update enemies
    for enemy in enemies:
        enemy.update(gun_pos[0], gun_pos[1])
        
        if enemy.check_collision_with_player(gun_pos[0], gun_pos[1]):
            life -= 1
            enemy.respawn()
            if life <= 0:
                game_over = True
    
    if bullets_missed >= MAX_BULLETS_MISSED:
        game_over = True
    
    # Cheat mode
    if cheat_mode:
        auto_rotate_angle += 6  # Even faster rotation
        if auto_rotate_angle >= 360:
            auto_rotate_angle = 0
        
        for enemy in enemies:
            angle_to_enemy = math.degrees(math.atan2(enemy.x - gun_pos[0], 
                                                      enemy.y - gun_pos[1]))
            angle_diff = abs(angle_to_enemy - auto_rotate_angle)
            
            if angle_diff < 5 or angle_diff > 355:
                fire_bullet()
                break


def move_gun_forward():
    if not game_over:
        new_x = gun_pos[0] + gun_speed * math.sin(math.radians(gun_angle))
        new_y = gun_pos[1] + gun_speed * math.cos(math.radians(gun_angle))
        
        if abs(new_x) < GRID_LENGTH - 50 and abs(new_y) < GRID_LENGTH - 50:
            gun_pos[0] = new_x
            gun_pos[1] = new_y


def move_gun_backward():
    if not game_over:
        new_x = gun_pos[0] - gun_speed * math.sin(math.radians(gun_angle))
        new_y = gun_pos[1] - gun_speed * math.cos(math.radians(gun_angle))
        
        if abs(new_x) < GRID_LENGTH - 50 and abs(new_y) < GRID_LENGTH - 50:
            gun_pos[0] = new_x
            gun_pos[1] = new_y


def rotate_gun_left():
    global gun_angle
    if not game_over and not cheat_mode:
        gun_angle += rotation_speed


def rotate_gun_right():
    global gun_angle
    if not game_over and not cheat_mode:
        gun_angle -= rotation_speed


def reset_game():
    global game_over, game_score, life, bullets_missed, bullets
    global gun_pos, gun_angle, cheat_mode, cheat_vision
    
    game_over = False
    game_score = 0
    life = 5
    bullets_missed = 0
    bullets = []
    gun_pos = [0, 0, 0]
    gun_angle = 0
    cheat_mode = False
    cheat_vision = False
    
    initialize_game()


def keyboardListener(key, x, y):
    global cheat_mode, cheat_vision, gun_angle, auto_rotate_angle
    
    # Move forward (W key)
    if key == b'w' or key == b'W':
        move_gun_forward()
    
    # Move backward (S key)
    if key == b's' or key == b'S':
        move_gun_backward()
    
    # Rotate gun left (A key)
    if key == b'a' or key == b'A':
        rotate_gun_left()
    
    # Rotate gun right (D key)
    if key == b'd' or key == b'D':
        rotate_gun_right()
    
    # Toggle cheat mode (C key)
    if key == b'c' or key == b'C':
        cheat_mode = not cheat_mode
        if cheat_mode:
            auto_rotate_angle = gun_angle
    
    # Toggle cheat vision (V key)
    if key == b'v' or key == b'V':
        if cheat_mode:
            cheat_vision = not cheat_vision
    
    # Reset the game if R key is pressed
    if key == b'r' or key == b'R':
        if game_over:
            reset_game()
    
    glutPostRedisplay()


def specialKeyListener(key, x, y):
    global camera_height, camera_angle
    
    if key == GLUT_KEY_UP:
        camera_height += 20
    
    if key == GLUT_KEY_DOWN:
        camera_height -= 20
        if camera_height < 100:
            camera_height = 100
    
    if key == GLUT_KEY_LEFT:
        camera_angle += 5
    
    if key == GLUT_KEY_RIGHT:
        camera_angle -= 5
    
    glutPostRedisplay()


def mouseListener(button, state, x, y):
    global first_person_mode
    
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN:
        if not game_over:
            fire_bullet()
    
    if button == GLUT_RIGHT_BUTTON and state == GLUT_DOWN:
        first_person_mode = not first_person_mode
    
    glutPostRedisplay()


def setupCamera():
    global gun_angle
    
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(fovY, 1.25, 0.1, 2000)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    if first_person_mode:
        if cheat_mode and cheat_vision:
            # Cheat vision: camera behind gun, follows auto-rotation
            cam_x = gun_pos[0] - 80 * math.sin(math.radians(auto_rotate_angle))
            cam_y = gun_pos[1] - 80 * math.cos(math.radians(auto_rotate_angle))
            cam_z = 70
            
            look_x = gun_pos[0] + 300 * math.sin(math.radians(auto_rotate_angle))
            look_y = gun_pos[1] + 300 * math.cos(math.radians(auto_rotate_angle))
            look_z = 50
        else:
            # Normal first-person: camera behind gun, follows gun angle
            cam_x = gun_pos[0] - 80 * math.sin(math.radians(gun_angle))
            cam_y = gun_pos[1] - 80 * math.cos(math.radians(gun_angle))
            cam_z = 70
            
            look_x = gun_pos[0] + 300 * math.sin(math.radians(gun_angle))
            look_y = gun_pos[1] + 300 * math.cos(math.radians(gun_angle))
            look_z = 50
        
        gluLookAt(cam_x, cam_y, cam_z,
                  look_x, look_y, look_z,
                  0, 0, 1)
    else:
        cam_x = camera_distance * math.sin(math.radians(camera_angle))
        cam_y = -camera_distance * math.cos(math.radians(camera_angle))
        cam_z = camera_height
        
        gluLookAt(cam_x, cam_y, cam_z,
                  0, 0, 0,
                  0, 0, 1)


def idle():
    update_game()
    
    if cheat_mode:
        global gun_angle
        gun_angle = auto_rotate_angle
    
    glutPostRedisplay()


def showScreen():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    glViewport(0, 0, 1000, 800)
    
    glEnable(GL_DEPTH_TEST)
    
    setupCamera()
    
    draw_grid()
    draw_player(lying_down=game_over)
    
    for enemy in enemies:
        enemy.draw(enemy_size_scale)
    
    for bullet in bullets:
        bullet.draw()
    
    draw_text(10, 770, f"Score: {game_score}")
    draw_text(10, 740, f"Life: {life}")
    draw_text(10, 710, f"Bullets Missed: {bullets_missed}/{MAX_BULLETS_MISSED}")
    
    if cheat_mode:
        draw_text(10, 680, "CHEAT MODE: ON")
    
    if first_person_mode:
        draw_text(10, 650, "Camera: First Person")
    else:
        draw_text(10, 650, "Camera: Third Person")
    
    if game_over:
        draw_text(400, 400, "GAME OVER!")
        draw_text(350, 370, "Press R to Restart")
    
    glutSwapBuffers()


def main():
    glutInit()
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
    glutInitWindowSize(1000, 800)
    glutInitWindowPosition(0, 0)
    wind = glutCreateWindow(b"Bullet Frenzy - 3D Game")
    
    glEnable(GL_DEPTH_TEST)
    glClearColor(0.1, 0.1, 0.15, 1)
    
    initialize_game()
    
    glutDisplayFunc(showScreen)
    glutKeyboardFunc(keyboardListener)
    glutSpecialFunc(specialKeyListener)
    glutMouseFunc(mouseListener)
    glutIdleFunc(idle)
    
    glutMainLoop()


if __name__ == "__main__":
    main()

NullFunctionError: Attempt to call an undefined function glutInit, check for bool(glutInit) before calling